In [1]:
#import keras
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
#from keras.optimizers import Adam 
#from tensorflow.keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras import applications
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from imutils import paths
import numpy as np
import cv2
import os
#import efficientnet.keras as efn
import tensorflow_addons as tfa
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()




In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)


# TRAIN_PATH = "/data/phuongnnq3/classify_receipt/data/train"
# VAL_PATH = "/data/phuongnnq3/classify_receipt/data/val"

TRAIN_PATH = "train"
VAL_PATH = "val"

L = ["cut-and-pair", "normal"]
totalTrain = len(list(paths.list_images(TRAIN_PATH)))
totalVal = len(list(paths.list_images(VAL_PATH)))
# path = "/data/phuongnnq3/classify_receipt/models/3_classes/1/"
path = "models/cut_pair_top_ef1/"
filepath = path+"model-{epoch:02d}-{val_accuracy:.3f}.h5"
# logdir="/data/phuongnnq3/classify_receipt/logs/3_classes/1/"
logdir="logs/cut_pair_top_ef1/"
#checkpoint = ModelCheckpoint(
#    filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=False)
tfboard = tf.keras.callbacks.TensorBoard(logdir)
BS = 8
img_size = (512, 128)# (960, 320)  # (img_height, img_width)
epochs = 10
n_class = 2

trainAug = ImageDataGenerator(rescale=1 / 255.0)

valAug = ImageDataGenerator(rescale=1 / 255.0)

trainGen = trainAug.flow_from_directory(
    TRAIN_PATH,
    class_mode="categorical",
    target_size=img_size,
    color_mode="rgb",
    shuffle=True,
    batch_size=BS)


valGen = valAug.flow_from_directory(
    VAL_PATH,
    class_mode="categorical",
    target_size=img_size,
    color_mode="rgb",
    shuffle=False,
    batch_size=BS)

#baseModel.trainable = False
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('test',
                                                class_mode="categorical",
                                                target_size=img_size,
                                                color_mode="rgb",
                                                shuffle=False,
                                                batch_size=BS)


cut_layer_name = 'block6a_activation'
# cut_layer_name = 'block6b_activation'
base_model = tf.keras.applications.EfficientNetB1(include_top=False, input_shape=(img_size[0], img_size[1], 3))
#base_model.trainable=False
x = base_model.get_layer(cut_layer_name).output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256)(x)
x = layers.ReLU()(x)
x = layers.Dense(128)(x)
x = layers.ReLU()(x)
out = layers.Dense(2, activation="softmax")(x)


#x = layers.Dense(256)(x)
#x = layers.Dropout(0.25)(x)
#x = layers.BatchNormalization()(x)
#out = layers.Dense(n_class, activation="softmax")(x)
model = tf.keras.Model(inputs=base_model.input, outputs=out)

#Build model

lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    1e-3, 1000, end_learning_rate=1e-5)
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              #optimizer=keras.optimizers.Adam(lr=0.00005),
              optimizer=opt,
              #metrics=[tfa.metrics.F1Score(num_classes=n_class, average='macro', threshold=0.5)])
              #metrics=[tf.keras.metrics.AUC()])
              metrics=[tf.keras.metrics.AUC()])
#base_model.summary()
#x = base_model.layers[-1].output
# x = base_model.get_layer(cut_layer_name).output
 
#x = layers.Flatten()(x)
#x = layers.Dense(512,activation = "relu")(x)

#x = layers.Dropout(0.1)(x)
#x = layers.Dense(512,"sigmoid")(x)
#x = layers.BatchNormalization()(x)
#out = layers.Dense(n_class, activation="softmax")(x)


#model = keras.Model(inputs=base_model.input, outputs=out)

model.summary()


# lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
#     1e-3, 1000, end_learning_rate=1e-5)
# opt = tf.keras.optimizers.RMSprop()

#model.compile(loss="categorical_crossentropy",
#              #optimizer=keras.optimizers.Adam(lr=0.00005),
#              optimizer="adam",
#              metrics=['accuracy'])
#              # metrics=['accuracy'])



Found 2143 images belonging to 2 classes.
Found 175 images belonging to 2 classes.
Found 717 images belonging to 2 classes.
Instructions for updating:
Colocations handled automatically by placer.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 128, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 512, 128, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 512, 128, 3)  7           rescaling[0][0]                  
________________________________________________________________________________________________

In [ ]:
H = model.fit(
    trainGen,
    steps_per_epoch=totalTrain // BS,
    validation_data=valGen,
    validation_steps=totalVal // BS,
    epochs=epochs,
    callbacks=[checkpoint, tfboard])

In [ ]:
results = cnn.evaluate(test_set, batch_size=BS)
print("test loss, test AUC:", results)